In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !python efficientad.py --dataset mvtec_ad --subdataset wood --train_steps 6000
!python '/content/drive/MyDrive/EfficientAD_/efficientad.py' --mvtec_ad_path "/content/drive/MyDrive/wood_otsu" --subdataset "wood" --train_steps 1500 -o "/content/drive/MyDrive/EfficientAD_/Deneme 5 Ağırlıklar"


Computing mean of features: 100% 63/63 [00:00<00:00, 138.61it/s]
Computing std of features: 100% 63/63 [00:00<00:00, 209.63it/s]
Current loss: 8.7328: 100% 1500/1500 [00:24<00:00, 61.60it/s]
Final map normalization: 100% 7/7 [00:00<00:00, 106.96it/s]
Final inference: 100% 140/140 [02:26<00:00,  1.05s/it]

🔍 IoU Threshold Optimization Results:
Threshold: 0.10 | Mean IoU: 0.2724
Threshold: 0.20 | Mean IoU: 0.1989
Threshold: 0.30 | Mean IoU: 0.1219
Threshold: 0.40 | Mean IoU: 0.0626
Threshold: 0.50 | Mean IoU: 0.0306
Threshold: 0.60 | Mean IoU: 0.0134
Threshold: 0.70 | Mean IoU: 0.0065
Threshold: 0.80 | Mean IoU: 0.0025
Threshold: 0.90 | Mean IoU: 0.0010

✅ Best IoU: 0.2724 @ Threshold = 0.10

✅ Anomaly Detection Sonucları
AUC Score          : 0.9171
F1 Score           : 0.8904
Precision          : 0.8553
Recall             : 0.9286
Optimal Threshold  : 0.1834
Confusion Matrix:
 [[59 11]
 [ 5 65]]

✅ Final Anomaly Detection Sonuçları
Final AUC Score        : 0.9171
Final F1 Score         

# Model Save

In [ ]:
import torch
import os
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm
from torchvision import transforms
from torch.serialization import add_safe_globals

# Güvenli sınıflar (PyTorch 2.6+ için)
add_safe_globals([
    torch.nn.Sequential,
    torch.nn.Conv2d,
    torch.nn.ReLU,
    torch.nn.Linear,
    torch.nn.BatchNorm2d,
    torch.nn.AvgPool2d,
    torch.nn.MaxPool2d,
    torch.nn.Flatten,
    transforms.Compose
])

# Yol ayarları
MODEL_SAVE_DIR = "/content/drive/MyDrive/EfficientAD_/Deneme 5 Ağırlıklar/trainings/mvtec_ad/wood"
TRAIN_DIR = "/content/drive/MyDrive/Wood_Dataset/wood/train"
save_path = os.path.join(MODEL_SAVE_DIR, "efficientad_complete_model.pth")

# Transform
image_size = 256
out_channels = 384
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Model ve quantile yükle
teacher = torch.load(os.path.join(MODEL_SAVE_DIR, 'teacher_final.pth'), weights_only=False)
student = torch.load(os.path.join(MODEL_SAVE_DIR, 'student_final.pth'), weights_only=False)
autoencoder = torch.load(os.path.join(MODEL_SAVE_DIR, 'autoencoder_final.pth'), weights_only=False)
quantiles = torch.load(os.path.join(MODEL_SAVE_DIR, 'map_quantiles_final.pth'), weights_only=False)

# ✅ float / numpy / tensor ayırt ederek tensor'a çevir
def ensure_tensor(v):
    if isinstance(v, torch.Tensor):
        return v
    elif isinstance(v, np.ndarray):
        return torch.from_numpy(v)
    elif isinstance(v, (float, int)):
        return torch.tensor(v)
    else:
        raise ValueError(f"Unsupported type in quantiles: {type(v)}")

map_quantiles = {k: ensure_tensor(v) for k, v in quantiles.items()}

# Normalizasyon hesapla
train_images = []
for ext in ["*.jpg", "*.png", "**/*.jpg", "**/*.png", "good/*.jpg", "good/*.png"]:
    train_images.extend(glob(os.path.join(TRAIN_DIR, ext), recursive=True))

if not train_images:
    print("No training images found. Using dummy stats.")
    channel_mean = torch.zeros((1, out_channels, 1, 1))
    channel_std = torch.ones((1, out_channels, 1, 1))
else:
    teacher_cpu = teacher.cpu()
    mean_outputs = []
    for img_path in tqdm(train_images[:50], desc="Computing mean"):
        try:
            img = transform(Image.open(img_path).convert("RGB")).unsqueeze(0)
            out = teacher_cpu(img)
            mean_outputs.append(out.mean(dim=[0, 2, 3]))
        except Exception as e:
            print(f"Mean error {img_path}: {e}")
    channel_mean = torch.mean(torch.stack(mean_outputs), dim=0)[None, :, None, None]

    std_outputs = []
    for img_path in tqdm(train_images[:50], desc="Computing std"):
        try:
            img = transform(Image.open(img_path).convert("RGB")).unsqueeze(0)
            out = teacher_cpu(img)
            std_outputs.append(((out - channel_mean)**2).mean(dim=[0, 2, 3]))
        except Exception as e:
            print(f"Std error {img_path}: {e}")
    channel_var = torch.mean(torch.stack(std_outputs), dim=0)[None, :, None, None]
    channel_std = torch.sqrt(channel_var)

# ➤ Tensor olarak kaydedilecek stat dict
normalization_params = {
    'mean': channel_mean,
    'std': channel_std
}

# Modeli kaydet
torch.save({
    "teacher": teacher.cpu(),
    "student": student.cpu(),
    "autoencoder": autoencoder.cpu(),
    "map_quantiles": map_quantiles,
    "normalization_params": normalization_params
}, save_path)

print("✅ efficientad_complete_model.pth başarıyla ve doğru formatta kaydedildi.")


No training images found. Using dummy stats.
✅ efficientad_complete_model.pth başarıyla ve doğru formatta kaydedildi.


# Modelin İçeriği

In [ ]:
import torch
import torchvision
import numpy as np
import collections
import pickle  # Bu önemli

path = "/content/drive/MyDrive/EfficientAD_/Deneme 5 Ağırlıklar/trainings/mvtec_ad/wood/efficientad_complete_model.pth"

# pickle_module=pickle ile yüklemeyi dene
data = torch.load(path, map_location="cpu", pickle_module=pickle)

print("Anahtarlar:", data.keys())  # dict_keys([...])

for key, value in data.items():
    print(f"\n🔑 {key} --> {type(value)}")



Anahtarlar: dict_keys(['teacher', 'student', 'autoencoder', 'map_quantiles', 'normalization_params'])

🔑 teacher --> <class 'torch.nn.modules.container.Sequential'>

🔑 student --> <class 'torch.nn.modules.container.Sequential'>

🔑 autoencoder --> <class 'torch.nn.modules.container.Sequential'>

🔑 map_quantiles --> <class 'dict'>

🔑 normalization_params --> <class 'dict'>


# Devamı

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm import tqdm
from torchvision import transforms
import tifffile

# Fix for PyTorch 2.6+ pickle loading issue
from torch.serialization import add_safe_globals
add_safe_globals([
    'torch.nn.modules.container.Sequential',
    'torch.nn.modules.conv.Conv2d',
    'torch.nn.modules.activation.ReLU',
    'torch.nn.modules.pooling.AvgPool2d',
    'torch.nn.modules.linear.Linear',
    'torch.nn.modules.normalization.GroupNorm',
    'torch.nn.modules.activation.GELU',
    'torchvision.models.resnet.Bottleneck',
    'collections.OrderedDict'
])

# Paths - UPDATE THESE TO MATCH YOUR ACTUAL DIRECTORIES
DATASET_PATH = "/content/drive/MyDrive/wood_otsu/wood/ground_truth"
MODEL_PATH = "/content/drive/MyDrive/EfficientAD_/result-2/trainings/mvtec_ad/wood"
OUTPUT_DIR = "/content/drive/MyDrive/EfficientAD_/result-ground_truth/png"
TRAIN_DIR = "/content/drive/MyDrive/Wood_Dataset/wood/train"  # Path to training images

# Print paths for verification
print(f"Dataset Path: {DATASET_PATH}")
print(f"Model Path: {MODEL_PATH}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Train Directory: {TRAIN_DIR}")

# Check if directories exist
for path, name in [(DATASET_PATH, "Dataset"), (MODEL_PATH, "Model"), (TRAIN_DIR, "Train")]:
    if not os.path.exists(path):
        print(f"WARNING: {name} path does not exist: {path}")
    else:
        print(f"{name} path exists: {path}")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Constants
on_gpu = torch.cuda.is_available()
out_channels = 384
image_size = 256

# Default transform for testing
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Loading models and quantiles...")
try:
    # Try loading with weights_only=False to handle PyTorch 2.6 changes
    teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_final.pth'), weights_only=False)
    student = torch.load(os.path.join(MODEL_PATH, 'student_final.pth'), weights_only=False)
    autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_final.pth'), weights_only=False)
    print("Successfully loaded final models")
except Exception as e:
    print(f"Error loading final models: {e}")
    try:
        # Try temporary models
        print("Trying temporary models...")
        teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_tmp.pth'), weights_only=False)
        student = torch.load(os.path.join(MODEL_PATH, 'student_tmp.pth'), weights_only=False)
        autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_tmp.pth'), weights_only=False)
        print("Successfully loaded temporary models")
    except Exception as e:
        print(f"Error loading temporary models: {e}")
        raise


# Load quantiles
print("Loading quantiles...")
try:
    quantiles = torch.load(os.path.join(MODEL_PATH, 'map_quantiles_final.pth'), weights_only=False)
    q_st_start = torch.tensor(quantiles['q_st_start']) # Convert to tensor
    q_st_end = torch.tensor(quantiles['q_st_end'])  # Convert to tensor
    q_ae_start = torch.tensor(quantiles['q_ae_start'])  # Convert to tensor
    q_ae_end = torch.tensor(quantiles['q_ae_end'])  # Convert to tensor
    print("Successfully loaded quantiles")
except Exception as e:
    print(f"Error loading quantiles: {e}")
    raise

if on_gpu:
    teacher = teacher.cuda()
    student = student.cuda()
    autoencoder = autoencoder.cuda()
    q_st_start = q_st_start.cuda()
    q_st_end = q_st_end.cuda()
    q_ae_start = q_ae_start.cuda()
    q_ae_end = q_ae_end.cuda()

# Set models to evaluation mode
teacher.eval()
student.eval()
autoencoder.eval()

# Load train data for computing teacher normalization
print("Loading training images for normalization...")

# Try different patterns to locate training images
train_patterns = [
    os.path.join(TRAIN_DIR, "*.jpg"),
    os.path.join(TRAIN_DIR, "*.png"),
    os.path.join(TRAIN_DIR, "**", "*.jpg"),
    os.path.join(TRAIN_DIR, "**", "*.png"),
    os.path.join(TRAIN_DIR, "good", "*.jpg"),
    os.path.join(TRAIN_DIR, "good", "*.png")
]

train_images = []
for pattern in train_patterns:
    found_images = glob(pattern, recursive=True)
    if found_images:
        print(f"Found {len(found_images)} images with pattern: {pattern}")
        train_images.extend(found_images)

if not train_images:
    print("ERROR: No training images found! Checking parent directories...")
    # Try to look in parent directories
    parent_dir = os.path.dirname(TRAIN_DIR)
    for pattern in [os.path.join(parent_dir, "**", "*.jpg"), os.path.join(parent_dir, "**", "*.png")]:
        found_images = glob(pattern, recursive=True)
        if found_images:
            print(f"Found {len(found_images)} images in parent directory with pattern: {pattern}")
            train_images.extend(found_images)

# Print first few train images for verification
if train_images:
    print(f"Total training images found: {len(train_images)}")
    print("First few training images:")
    for i in range(min(5, len(train_images))):
        print(f"  - {train_images[i]}")
else:
    print("ERROR: Could not find any training images! Cannot proceed without training images.")
    # Instead of crashing, we'll use pre-computed statistics from the quantiles
    print("Using pre-computed statistics from quantiles to continue...")
    # Create some dummy statistics that should work reasonably well
    channel_mean = torch.zeros((1, out_channels, 1, 1))
    channel_std = torch.ones((1, out_channels, 1, 1))
    if on_gpu:
        channel_mean = channel_mean.cuda()
        channel_std = channel_std.cuda()

# If we have training images, compute normalization statistics
if train_images:
    # If there are too many images, use just a subset for normalization
    if len(train_images) > 50:
        import random
        train_images = random.sample(train_images, 50)
        print(f"Using random subset of 50 images for normalization")

    # Compute teacher normalization
    print("Computing teacher normalization...")
    mean_outputs = []
    with torch.no_grad():
        for img_path in tqdm(train_images):
            try:
                image = Image.open(img_path).convert('RGB')
                image = transform(image).unsqueeze(0)
                if on_gpu:
                    image = image.cuda()
                teacher_output = teacher(image)
                mean_output = torch.mean(teacher_output, dim=[0, 2, 3])
                mean_outputs.append(mean_output)
            except Exception as e:
                print(f"Error processing training image {img_path}: {e}")

    if not mean_outputs:
        print("ERROR: Failed to process any training images!")
        # Use dummy statistics
        channel_mean = torch.zeros((1, out_channels, 1, 1))
        channel_std = torch.ones((1, out_channels, 1, 1))
        if on_gpu:
            channel_mean = channel_mean.cuda()
            channel_std = channel_std.cuda()
    else:
        channel_mean = torch.mean(torch.stack(mean_outputs), dim=0)
        channel_mean = channel_mean[None, :, None, None]

        mean_distances = []
        with torch.no_grad():
            for img_path in tqdm(train_images):
                try:
                    image = Image.open(img_path).convert('RGB')
                    image = transform(image).unsqueeze(0)
                    if on_gpu:
                        image = image.cuda()
                    teacher_output = teacher(image)
                    distance = (teacher_output - channel_mean) ** 2
                    mean_distance = torch.mean(distance, dim=[0, 2, 3])
                    mean_distances.append(mean_distance)
                except Exception as e:
                    print(f"Error processing training image distance {img_path}: {e}")

        channel_var = torch.mean(torch.stack(mean_distances), dim=0)
        channel_var = channel_var[None, :, None, None]
        channel_std = torch.sqrt(channel_var)

# Find all test images (both good and defect)
print("Looking for test images...")
test_patterns = [
    os.path.join(DATASET_PATH, "**", "*.jpg"),
    os.path.join(DATASET_PATH, "**", "*.png"),
    os.path.join(DATASET_PATH, "*.jpg"),
    os.path.join(DATASET_PATH, "*.png"),
    os.path.join(DATASET_PATH, "good", "*.jpg"),
    os.path.join(DATASET_PATH, "good", "*.png"),
    os.path.join(DATASET_PATH, "defect", "*.jpg"),
    os.path.join(DATASET_PATH, "defect", "*.png")
]

test_images = []
for pattern in test_patterns:
    found_images = glob(pattern, recursive=True)
    if found_images:
        print(f"Found {len(found_images)} test images with pattern: {pattern}")
        test_images.extend(found_images)

# Remove duplicates
test_images = list(set(test_images))

if not test_images:
    print("ERROR: No test images found!")
    exit(1)

print(f"Found {len(test_images)} total test images.")
print("First few test images:")
for i in range(min(5, len(test_images))):
    print(f"  - {test_images[i]}")

# Run inference on all test images
print("Starting inference...")
with torch.no_grad():
    for img_path in tqdm(test_images, desc="Processing test images"):
        try:
            # Extract class and filename
            relative_path = os.path.relpath(img_path, DATASET_PATH)
            parts = relative_path.split(os.sep)

            if len(parts) > 1:
                class_name = parts[0]
                filename = parts[1]
            else:
                class_name = "unknown"
                filename = parts[0]

            img_id = os.path.splitext(filename)[0]

            # Create output directory for this class
            class_output_dir = os.path.join(OUTPUT_DIR, class_name)
            os.makedirs(class_output_dir, exist_ok=True)

            # Load and preprocess image
            image = Image.open(img_path).convert('RGB')
            orig_width, orig_height = image.size
            image_tensor = transform(image).unsqueeze(0)

            if on_gpu:
                image_tensor = image_tensor.cuda()

            # Run EfficientAD inference
            teacher_output = teacher(image_tensor)
            teacher_output = (teacher_output - channel_mean) / channel_std
            student_output = student(image_tensor)
            autoencoder_output = autoencoder(image_tensor)

            map_st = torch.mean((teacher_output - student_output[:, :out_channels])**2,
                               dim=1, keepdim=True)
            map_ae = torch.mean((autoencoder_output - student_output[:, out_channels:])**2,
                               dim=1, keepdim=True)

            # Apply quantile normalization
            map_st = 0.1 * (map_st - q_st_start) / (q_st_end - q_st_start)
            map_ae = 0.1 * (map_ae - q_ae_start) / (q_ae_end - q_ae_start)

            # Combine maps
            map_combined = 0.5 * map_st + 0.5 * map_ae

            # Pad and resize back to original dimensions
            map_combined = torch.nn.functional.pad(map_combined, (4, 4, 4, 4))
            map_combined = torch.nn.functional.interpolate(
                map_combined, (orig_height, orig_width), mode='bilinear')

            # Convert to numpy
            anomaly_map = map_combined[0, 0].cpu().numpy()

            # Save as tiff file (for full precision)
            tiff_path = os.path.join(class_output_dir, f"{img_id}.tiff")
            tifffile.imwrite(tiff_path, anomaly_map)

            # Also save a visualization
            plt.figure(figsize=(10, 6))

            # Original image
            plt.subplot(1, 2, 1)
            plt.imshow(np.array(Image.open(img_path).convert('RGB')))
            plt.title("Original Image")
            plt.axis('off')

            # Anomaly map
            plt.subplot(1, 2, 2)
            plt.imshow(anomaly_map, cmap='jet')
            plt.title(f"Anomaly Map (Max: {np.max(anomaly_map):.3f})")
            plt.axis('off')

            # Save visualization
            vis_path = os.path.join(class_output_dir, f"{img_id}_vis.png")
            plt.savefig(vis_path, bbox_inches='tight')
            plt.close()
        except Exception as e:
            print(f"Error processing test image {img_path}: {e}")

print(f"All anomaly maps saved to {OUTPUT_DIR}")
print("Done!")

Dataset Path: /content/drive/MyDrive/wood_otsu/wood/ground_truth
Model Path: /content/drive/MyDrive/EfficientAD_/result-2/trainings/mvtec_ad/wood
Output Directory: /content/drive/MyDrive/EfficientAD_/result-ground_truth/png
Train Directory: /content/drive/MyDrive/Wood_Dataset/wood/train
Dataset path exists: /content/drive/MyDrive/wood_otsu/wood/ground_truth
Model path exists: /content/drive/MyDrive/EfficientAD_/result-2/trainings/mvtec_ad/wood
Train path exists: /content/drive/MyDrive/Wood_Dataset/wood/train
Loading models and quantiles...
Successfully loaded final models
Loading quantiles...
Successfully loaded quantiles
Loading training images for normalization...
Found 20 images with pattern: /content/drive/MyDrive/Wood_Dataset/wood/train/**/*.jpg
Found 20 images with pattern: /content/drive/MyDrive/Wood_Dataset/wood/train/good/*.jpg
Total training images found: 40
First few training images:
  - /content/drive/MyDrive/Wood_Dataset/wood/train/good/8.jpg
  - /content/drive/MyDrive/Woo

100%|██████████| 40/40 [00:01<00:00, 24.58it/s]


Looking for test images...
Found 71 test images with pattern: /content/drive/MyDrive/wood_otsu/wood/ground_truth/**/*.jpg
Found 71 test images with pattern: /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/*.jpg
Found 71 total test images.
First few test images:
  - /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/101900007_mask.jpg
  - /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/100100034_mask.jpg
  - /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/100100026_mask.jpg
  - /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/101900017_mask.jpg
  - /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/101900010_mask.jpg
Starting inference...


Processing test images: 100%|██████████| 71/71 [01:21<00:00,  1.15s/it]

All anomaly maps saved to /content/drive/MyDrive/EfficientAD_/result-ground_truth/png
Done!


In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm import tqdm
from torchvision import transforms
import tifffile
import cv2

# Fix for PyTorch 2.6+ pickle loading issue
from torch.serialization import add_safe_globals
add_safe_globals([
    'torch.nn.modules.container.Sequential',
    'torch.nn.modules.conv.Conv2d',
    'torch.nn.modules.activation.ReLU',
    'torch.nn.modules.pooling.AvgPool2d',
    'torch.nn.modules.linear.Linear',
    'torch.nn.modules.normalization.GroupNorm',
    'torch.nn.modules.activation.GELU',
    'torchvision.models.resnet.Bottleneck',
    'collections.OrderedDict'
])

# Paths - UPDATE THESE TO MATCH YOUR ACTUAL DIRECTORIES

DATASET_PATH = "/content/drive/MyDrive/wood_otsu/wood/test"
MODEL_PATH = "/content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood"
OUTPUT_DIR = "/content/drive/MyDrive/EfficientAD_/result-13Mayıs_/png"
TRAIN_DIR = "/content/drive/MyDrive/Wood_Dataset/wood/train"  # Path to training images
GT_DIR = "/content/drive/MyDrive/wood_otsu/wood/ground_truth"  # Ground truth directory

# Contour threshold for anomaly detection - Updated to 0.250
CONTOUR_THRESHOLD = 0.250

# Print paths for verification
print(f"Dataset Path: {DATASET_PATH}")
print(f"Model Path: {MODEL_PATH}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Train Directory: {TRAIN_DIR}")
print(f"Ground Truth Directory: {GT_DIR}")
print(f"Contour Threshold: {CONTOUR_THRESHOLD}")

# Check if directories exist
for path, name in [(DATASET_PATH, "Dataset"), (MODEL_PATH, "Model"), (TRAIN_DIR, "Train"), (GT_DIR, "Ground Truth")]:
    if not os.path.exists(path):
        print(f"WARNING: {name} path does not exist: {path}")
    else:
        print(f"{name} path exists: {path}")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "grid_views"), exist_ok=True)

# Constants
on_gpu = torch.cuda.is_available()
out_channels = 384
image_size = 256

# Default transform for testing
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Helper functions for contour detection and border removal
def filter_contours(contours, image_shape, min_area=100, border_margin=30):
    """Filter contours based on size and position"""
    h, w = image_shape[:2]
    filtered = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, cw, ch = cv2.boundingRect(cnt)
        if area > min_area and border_margin < x < w-border_margin and border_margin < y < h-border_margin:
            filtered.append(cnt)
    return filtered

def remove_border_artifacts(anomaly_map, border_px=30):
    """Remove border artifacts from anomaly map"""
    h, w = anomaly_map.shape
    mask = np.zeros_like(anomaly_map)
    mask[border_px:h-border_px, border_px:w-border_px] = 1
    return anomaly_map * mask

print("Loading models and quantiles...")
try:
    # Try loading with weights_only=False to handle PyTorch 2.6 changes
    teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_final.pth'), weights_only=False)
    student = torch.load(os.path.join(MODEL_PATH, 'student_final.pth'), weights_only=False)
    autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_final.pth'), weights_only=False)
    print("Successfully loaded final models")
except Exception as e:
    print(f"Error loading final models: {e}")
    try:
        # Try temporary models
        print("Trying temporary models...")
        teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_tmp.pth'), weights_only=False)
        student = torch.load(os.path.join(MODEL_PATH, 'student_tmp.pth'), weights_only=False)
        autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_tmp.pth'), weights_only=False)
        print("Successfully loaded temporary models")
    except Exception as e:
        print(f"Error loading temporary models: {e}")
        raise

# Load quantiles
print("Loading quantiles...")
try:
    quantiles = torch.load(os.path.join(MODEL_PATH, 'map_quantiles_final.pth'), weights_only=False)
    q_st_start = torch.tensor(quantiles['q_st_start']) # Convert to tensor
    q_st_end = torch.tensor(quantiles['q_st_end'])  # Convert to tensor
    q_ae_start = torch.tensor(quantiles['q_ae_start'])  # Convert to tensor
    q_ae_end = torch.tensor(quantiles['q_ae_end'])  # Convert to tensor
    print("Successfully loaded quantiles")
except Exception as e:
    print(f"Error loading quantiles: {e}")
    raise

if on_gpu:
    teacher = teacher.cuda()
    student = student.cuda()
    autoencoder = autoencoder.cuda()
    q_st_start = q_st_start.cuda()
    q_st_end = q_st_end.cuda()
    q_ae_start = q_ae_start.cuda()
    q_ae_end = q_ae_end.cuda()

# Set models to evaluation mode
teacher.eval()
student.eval()
autoencoder.eval()

# Load train data for computing teacher normalization
print("Loading training images for normalization...")

# Try different patterns to locate training images
train_patterns = [
    os.path.join(TRAIN_DIR, "*.jpg"),
    os.path.join(TRAIN_DIR, "*.png"),
    os.path.join(TRAIN_DIR, "**", "*.jpg"),
    os.path.join(TRAIN_DIR, "**", "*.png"),
    os.path.join(TRAIN_DIR, "good", "*.jpg"),
    os.path.join(TRAIN_DIR, "good", "*.png")
]

train_images = []
for pattern in train_patterns:
    found_images = glob(pattern, recursive=True)
    if found_images:
        print(f"Found {len(found_images)} images with pattern: {pattern}")
        train_images.extend(found_images)

if not train_images:
    print("ERROR: No training images found! Checking parent directories...")
    # Try to look in parent directories
    parent_dir = os.path.dirname(TRAIN_DIR)
    for pattern in [os.path.join(parent_dir, "**", "*.jpg"), os.path.join(parent_dir, "**", "*.png")]:
        found_images = glob(pattern, recursive=True)
        if found_images:
            print(f"Found {len(found_images)} images in parent directory with pattern: {pattern}")
            train_images.extend(found_images)

# Print first few train images for verification
if train_images:
    print(f"Total training images found: {len(train_images)}")
    print("First few training images:")
    for i in range(min(5, len(train_images))):
        print(f"  - {train_images[i]}")
else:
    print("ERROR: Could not find any training images! Cannot proceed without training images.")
    # Instead of crashing, we'll use pre-computed statistics from the quantiles
    print("Using pre-computed statistics from quantiles to continue...")
    # Create some dummy statistics that should work reasonably well
    channel_mean = torch.zeros((1, out_channels, 1, 1))
    channel_std = torch.ones((1, out_channels, 1, 1))
    if on_gpu:
        channel_mean = channel_mean.cuda()
        channel_std = channel_std.cuda()

# If we have training images, compute normalization statistics
if train_images:
    # If there are too many images, use just a subset for normalization
    if len(train_images) > 50:
        import random
        train_images = random.sample(train_images, 50)
        print(f"Using random subset of 50 images for normalization")

    # Compute teacher normalization
    print("Computing teacher normalization...")
    mean_outputs = []
    with torch.no_grad():
        for img_path in tqdm(train_images):
            try:
                image = Image.open(img_path).convert('RGB')
                image = transform(image).unsqueeze(0)
                if on_gpu:
                    image = image.cuda()
                teacher_output = teacher(image)
                mean_output = torch.mean(teacher_output, dim=[0, 2, 3])
                mean_outputs.append(mean_output)
            except Exception as e:
                print(f"Error processing training image {img_path}: {e}")

    if not mean_outputs:
        print("ERROR: Failed to process any training images!")
        # Use dummy statistics
        channel_mean = torch.zeros((1, out_channels, 1, 1))
        channel_std = torch.ones((1, out_channels, 1, 1))
        if on_gpu:
            channel_mean = channel_mean.cuda()
            channel_std = channel_std.cuda()
    else:
        channel_mean = torch.mean(torch.stack(mean_outputs), dim=0)
        channel_mean = channel_mean[None, :, None, None]

        mean_distances = []
        with torch.no_grad():
            for img_path in tqdm(train_images):
                try:
                    image = Image.open(img_path).convert('RGB')
                    image = transform(image).unsqueeze(0)
                    if on_gpu:
                        image = image.cuda()
                    teacher_output = teacher(image)
                    distance = (teacher_output - channel_mean) ** 2
                    mean_distance = torch.mean(distance, dim=[0, 2, 3])
                    mean_distances.append(mean_distance)
                except Exception as e:
                    print(f"Error processing training image distance {img_path}: {e}")

        channel_var = torch.mean(torch.stack(mean_distances), dim=0)
        channel_var = channel_var[None, :, None, None]
        channel_std = torch.sqrt(channel_var)

# Find all test images (both good and defect)
print("Looking for test images...")
test_patterns = [
    os.path.join(DATASET_PATH, "**", "*.jpg"),
    os.path.join(DATASET_PATH, "**", "*.png"),
    os.path.join(DATASET_PATH, "*.jpg"),
    os.path.join(DATASET_PATH, "*.png"),
    os.path.join(DATASET_PATH, "good", "*.jpg"),
    os.path.join(DATASET_PATH, "good", "*.png"),
    os.path.join(DATASET_PATH, "defect", "*.jpg"),
    os.path.join(DATASET_PATH, "defect", "*.png")
]

test_images = []
for pattern in test_patterns:
    found_images = glob(pattern, recursive=True)
    if found_images:
        print(f"Found {len(found_images)} test images with pattern: {pattern}")
        test_images.extend(found_images)

# Remove duplicates
test_images = list(set(test_images))

if not test_images:
    print("ERROR: No test images found!")
    exit(1)

print(f"Found {len(test_images)} total test images.")
print("First few test images:")
for i in range(min(5, len(test_images))):
    print(f"  - {test_images[i]}")

# Run inference on all test images
print("Starting inference...")
results = []

with torch.no_grad():
    for img_path in tqdm(test_images, desc="Processing test images"):
        try:
            # Extract class and filename
            relative_path = os.path.relpath(img_path, DATASET_PATH)
            parts = relative_path.split(os.sep)

            if len(parts) > 1:
                class_name = parts[0]
                filename = parts[1]
            else:
                class_name = "unknown"
                filename = parts[0]

            img_id = os.path.splitext(filename)[0]

            # Create output directory for this class
            class_output_dir = os.path.join(OUTPUT_DIR, class_name)
            os.makedirs(class_output_dir, exist_ok=True)

            # Load and preprocess image
            image = Image.open(img_path).convert('RGB')
            orig_width, orig_height = image.size
            image_tensor = transform(image).unsqueeze(0)

            if on_gpu:
                image_tensor = image_tensor.cuda()

            # Run EfficientAD inference
            teacher_output = teacher(image_tensor)
            teacher_output = (teacher_output - channel_mean) / channel_std
            student_output = student(image_tensor)
            autoencoder_output = autoencoder(image_tensor)

            map_st = torch.mean((teacher_output - student_output[:, :out_channels])**2,
                               dim=1, keepdim=True)
            map_ae = torch.mean((autoencoder_output - student_output[:, out_channels:])**2,
                               dim=1, keepdim=True)

            # Apply quantile normalization
            map_st = 0.1 * (map_st - q_st_start) / (q_st_end - q_st_start)
            map_ae = 0.1 * (map_ae - q_ae_start) / (q_ae_end - q_ae_start)

            # Combine maps
            map_combined = 0.5 * map_st + 0.5 * map_ae

            # Pad and resize back to original dimensions
            map_combined = torch.nn.functional.pad(map_combined, (4, 4, 4, 4))
            map_combined = torch.nn.functional.interpolate(
                map_combined, (orig_height, orig_width), mode='bilinear')

            # Convert to numpy
            anomaly_map = map_combined[0, 0].cpu().numpy()

            # Save as tiff file (for full precision)
            tiff_path = os.path.join(class_output_dir, f"{img_id}.tiff")
            tifffile.imwrite(tiff_path, anomaly_map)

            # Load original image for overlay
            orig_img = np.array(Image.open(img_path).convert('RGB'))

            # Remove border artifacts for scoring
            anomaly_map_masked = remove_border_artifacts(anomaly_map.copy(), border_px=40)
            max_score = np.max(anomaly_map_masked)

            # Create overlay with contours
            overlay_img = orig_img.copy()

            # Create binary mask for contours
            binary_mask = (anomaly_map_masked > CONTOUR_THRESHOLD).astype(np.uint8) * 255
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = filter_contours(contours, orig_img.shape, min_area=200, border_margin=40)

            # Draw contours on overlay
            overlay_img_cv = cv2.cvtColor(overlay_img, cv2.COLOR_RGB2BGR)
            cv2.drawContours(overlay_img_cv, contours, -1, (0, 0, 255), 2)  # Red contours

            # Add prediction text
            prediction = "Anomali" if max_score > CONTOUR_THRESHOLD else "Normal"
            cv2.putText(overlay_img_cv, f"Tahmin: {prediction} ({max_score:.3f})",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            # Convert back to RGB for matplotlib
            overlay_img = cv2.cvtColor(overlay_img_cv, cv2.COLOR_BGR2RGB)

            # Create heatmap visualization
            tiff_norm = cv2.normalize(anomaly_map, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
            tiff_colored = cv2.applyColorMap(tiff_norm, cv2.COLORMAP_JET)
            tiff_colored = cv2.cvtColor(tiff_colored, cv2.COLOR_BGR2RGB)

            # Try to load ground truth mask if available
            gt_mask = None
            potential_gt_paths = [
                os.path.join(GT_DIR, class_name, f"{img_id}_mask.png"),
                os.path.join(GT_DIR, class_name, f"{img_id}_mask.jpg"),
                os.path.join(GT_DIR, "defect", f"{img_id}_mask.png"),
                os.path.join(GT_DIR, "defect", f"{img_id}_mask.jpg"),
                os.path.join(GT_DIR, "ground_truth", class_name, f"{img_id}_mask.png"),
                os.path.join(GT_DIR, "ground_truth", class_name, f"{img_id}_mask.jpg"),
                os.path.join(GT_DIR, f"{img_id}_mask.png"),
                os.path.join(GT_DIR, f"{img_id}_mask.jpg")
            ]

            for gt_path in potential_gt_paths:
                if os.path.exists(gt_path):
                    gt_mask = np.array(Image.open(gt_path).convert('RGB'))
                    print(f"Found ground truth mask at: {gt_path}")
                    break

            # If ground truth mask not found, create a blank one
            if gt_mask is None:
                gt_mask = np.ones_like(orig_img) * 255
                # Only center part is black
                cv2.rectangle(gt_mask, (0, 0), (orig_width, orig_height), (0, 0, 0), -1)
                gt_mask[10:-10, 10:-10] = [255, 255, 255]

            # Create grid visualization (Original, Anomaly Map, Overlay, Ground Truth)
            plt.figure(figsize=(15, 5))

            # Original Image
            plt.subplot(1, 4, 1)
            plt.imshow(orig_img)
            plt.title(f"{filename} ({class_name}) - Original")
            plt.axis('off')

            # Anomaly Map
            plt.subplot(1, 4, 2)
            plt.imshow(tiff_colored)
            plt.title(f"Anomaly Map (.tiff) - Max: {max_score:.3f}")
            plt.axis('off')

            # Overlay
            plt.subplot(1, 4, 3)
            plt.imshow(overlay_img)
            plt.title(f"Overlay (Tahmin: {prediction})")
            plt.axis('off')

            # Ground Truth
            plt.subplot(1, 4, 4)
            plt.imshow(gt_mask)
            plt.title("Ground Truth")
            plt.axis('off')

            # Save grid visualization
            grid_path = os.path.join(OUTPUT_DIR, "grid_views", f"{class_name}_{img_id}_grid.png")
            plt.tight_layout()
            plt.savefig(grid_path, dpi=150, bbox_inches='tight')
            plt.close()

            # Store result for summary
            results.append({
                'path': img_path,
                'class': class_name,
                'filename': filename,
                'img_id': img_id,
                'score': max_score,
                'prediction': prediction,
                'has_contours': len(contours) > 0,
                'grid_path': grid_path
            })

        except Exception as e:
            print(f"Error processing test image {img_path}: {e}")
            import traceback
            traceback.print_exc()

# Create summary report
try:
    print("Creating summary report...")
    report_path = os.path.join(OUTPUT_DIR, "summary_report.html")
    with open(report_path, 'w') as f:
        f.write("<html><head>")
        f.write("<style>")
        f.write("body { font-family: Arial, sans-serif; margin: 20px; }")
        f.write("h1 { color: #333; }")
        f.write("table { border-collapse: collapse; width: 100%; }")
        f.write("th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }")
        f.write("th { background-color: #f2f2f2; }")
        f.write("tr:nth-child(even) { background-color: #f9f9f9; }")
        f.write("img { max-width: 1000px; border: 1px solid #ddd; }")
        f.write("</style>")
        f.write("</head><body>")
        f.write("<h1>Anomaly Detection Results</h1>")

        # Summary statistics
        good_samples = [r for r in results if r['class'] == 'good']
        defect_samples = [r for r in results if r['class'] != 'good']

        f.write(f"<p>Total samples: {len(results)}</p>")
        f.write(f"<p>Good samples: {len(good_samples)}</p>")
        f.write(f"<p>Defect samples: {len(defect_samples)}</p>")

        # Table of results
        f.write("<h2>Results Table</h2>")
        f.write("<table>")
        f.write("<tr><th>ID</th><th>Class</th><th>Score</th><th>Prediction</th></tr>")

        for r in sorted(results, key=lambda x: x['score'], reverse=True):
            f.write(f"<tr>")
            f.write(f"<td>{r['img_id']}</td>")
            f.write(f"<td>{r['class']}</td>")
            f.write(f"<td>{r['score']:.4f}</td>")
            f.write(f"<td>{r['prediction']}</td>")
            f.write(f"</tr>")

        f.write("</table>")

        # Images
        f.write("<h2>Top Anomalies</h2>")

        # Sort by score and take top 20
        top_anomalies = sorted(results, key=lambda x: x['score'], reverse=True)[:20]

        for r in top_anomalies:
            f.write(f"<h3>{r['filename']} (Class: {r['class']}, Score: {r['score']:.4f})</h3>")
            f.write(f"<img src='{os.path.relpath(r['grid_path'], OUTPUT_DIR)}'>")
            f.write("<hr>")

        f.write("</body></html>")

    print(f"Summary report created at: {report_path}")
except Exception as e:
    print(f"Error creating summary report: {e}")

print(f"All anomaly maps and visualizations saved to {OUTPUT_DIR}")
print("Done!")

Dataset Path: /content/drive/MyDrive/wood_otsu/wood/test
Model Path: /content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood
Output Directory: /content/drive/MyDrive/EfficientAD_/result-final_/png
Train Directory: /content/drive/MyDrive/Wood_Dataset/wood/train
Ground Truth Directory: /content/drive/MyDrive/wood_otsu/wood/ground_truth
Contour Threshold: 0.25
Dataset path exists: /content/drive/MyDrive/wood_otsu/wood/test
Model path exists: /content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood
Train path exists: /content/drive/MyDrive/Wood_Dataset/wood/train
Ground Truth path exists: /content/drive/MyDrive/wood_otsu/wood/ground_truth
Loading models and quantiles...
Successfully loaded final models
Loading quantiles...
Successfully loaded quantiles
Loading training images for normalization...
Found 20 images with pattern: /content/drive/MyDrive/Wood_Dataset/wood/train/**/*.jpg
Found 20 images with pattern: /content/drive/MyDrive/Wood_Dat

100%|██████████| 40/40 [00:01<00:00, 24.84it/s]


Looking for test images...
Found 140 test images with pattern: /content/drive/MyDrive/wood_otsu/wood/test/**/*.jpg
Found 70 test images with pattern: /content/drive/MyDrive/wood_otsu/wood/test/good/*.jpg
Found 70 test images with pattern: /content/drive/MyDrive/wood_otsu/wood/test/defect/*.jpg
Found 140 total test images.
First few test images:
  - /content/drive/MyDrive/wood_otsu/wood/test/good/113.jpg
  - /content/drive/MyDrive/wood_otsu/wood/test/defect/101100012.jpg
  - /content/drive/MyDrive/wood_otsu/wood/test/defect/100000000.jpg
  - /content/drive/MyDrive/wood_otsu/wood/test/defect/101500001.jpg
  - /content/drive/MyDrive/wood_otsu/wood/test/good/102.jpg
Starting inference...


Processing test images:   1%|          | 1/140 [00:00<01:39,  1.39it/s]

Found ground truth mask at: /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/101100012_mask.jpg


...

Found ground truth mask at: /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/100500011_mask.jpg


Processing test images:  99%|█████████▉| 139/140 [01:52<00:00,  1.17it/s]

Found ground truth mask at: /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/101900017_mask.jpg


Processing test images: 100%|██████████| 140/140 [01:53<00:00,  1.24it/s]

Creating summary report...
Summary report created at: /content/drive/MyDrive/EfficientAD_/result-final_/png/summary_report.html
All anomaly maps and visualizations saved to /content/drive/MyDrive/EfficientAD_/result-final_/png
Done!


In [ ]:
# prompt: modeli kaydeder misin arayüz yaparken kullanmam gerekcek

import os
import torch
import numpy as np # Needed for dummy variables if training images are not found
from PIL import Image # Needed for loading dummy images if needed
from torchvision import transforms # Needed for loading dummy transform if needed
from glob import glob # Needed for finding training images
from tqdm import tqdm # Needed for progress bar during normalization computation

# Fix for PyTorch 2.6+ pickle loading issue - Ensure this is also in the saving cell
from torch.serialization import add_safe_globals
add_safe_globals([
    'torch.nn.modules.container.Sequential',
    'torch.nn.modules.conv.Conv2d',
    'torch.nn.modules.activation.ReLU',
    'torch.nn.modules.pooling.AvgPool2d',
    'torch.nn.modules.linear.Linear',
    'torch.nn.modules.normalization.GroupNorm',
    'torch.nn.modules.activation.GELU',
    'torchvision.models.resnet.Bottleneck',
    'collections.OrderedDict'
])


# Define paths again to make sure they are available in this cell
MODEL_PATH = "/content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood" # Replace with the correct model path
TRAIN_DIR = "/content/drive/MyDrive/Wood_Dataset/wood/train"  # Path to training images

# Constants (ensure these are also defined)
on_gpu = torch.cuda.is_available()
out_channels = 384
image_size = 256

# Default transform (ensure this is also defined)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Loading models and quantiles for saving...")
try:
    # Try loading with weights_only=False to handle PyTorch 2.6 changes
    teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_final.pth'), weights_only=False)
    student = torch.load(os.path.join(MODEL_PATH, 'student_final.pth'), weights_only=False)
    autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_final.pth'), weights_only=False)
    print("Successfully loaded final models")
except Exception as e:
    print(f"Error loading final models: {e}")
    try:
        # Try temporary models
        print("Trying temporary models...")
        teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_tmp.pth'), weights_only=False)
        student = torch.load(os.path.join(MODEL_PATH, 'student_tmp.pth'), weights_only=False)
        autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_tmp.pth'), weights_only=False)
        print("Successfully loaded temporary models")
    except Exception as e:
        print(f"Error loading temporary models: {e}")
        raise


# Load quantiles
print("Loading quantiles for saving...")
try:
    quantiles = torch.load(os.path.join(MODEL_PATH, 'map_quantiles_final.pth'), weights_only=False)
    q_st_start = torch.tensor(quantiles['q_st_start']) # Convert to tensor
    q_st_end = torch.tensor(quantiles['q_st_end'])  # Convert to tensor
    q_ae_start = torch.tensor(quantiles['q_ae_start'])  # Convert to tensor
    q_ae_end = torch.tensor(quantiles['q_ae_end'])  # Convert to tensor
    print("Successfully loaded quantiles")
except Exception as e:
    print(f"Error loading quantiles: {e}")
    raise

if on_gpu:
    teacher = teacher.cuda()
    student = student.cuda()
    autoencoder = autoencoder.cuda()
    q_st_start = q_st_start.cuda()
    q_st_end = q_st_end.cuda()
    q_ae_start = q_ae_start.cuda()
    q_ae_end = q_ae_end.cuda()

# Set models to evaluation mode (important for consistent behavior)
teacher.eval()
student.eval()
autoencoder.eval()

# Load train data for computing teacher normalization if not already computed
# This part is crucial to ensure channel_mean and channel_std are defined
print("Loading training images for normalization (if needed)...")

train_patterns = [
    os.path.join(TRAIN_DIR, "*.jpg"),
    os.path.join(TRAIN_DIR, "*.png"),
    os.path.join(TRAIN_DIR, "**", "*.jpg"),
    os.path.join(TRAIN_DIR, "**", "*.png"),
    os.path.join(TRAIN_DIR, "good", "*.jpg"),
    os.path.join(TRAIN_DIR, "good", "*.png")
]

train_images = []
for pattern in train_patterns:
    found_images = glob(pattern, recursive=True)
    if found_images:
        train_images.extend(found_images)

if not train_images:
    print("WARNING: No training images found. Using dummy statistics.")
    # Create some dummy statistics that should work reasonably well
    channel_mean = torch.zeros((1, out_channels, 1, 1))
    channel_std = torch.ones((1, out_channels, 1, 1))
    if on_gpu:
        channel_mean = channel_mean.cuda()
        channel_std = channel_std.cuda()
else:
    # If we have training images, compute normalization statistics
    print("Computing teacher normalization...")
    mean_outputs = []
    with torch.no_grad():
        # Limit training images for speed if too many
        sample_images = train_images[:min(50, len(train_images))] # Use a subset
        for img_path in tqdm(sample_images, desc="Computing mean"):
             try:
                image = Image.open(img_path).convert('RGB')
                image = transform(image).unsqueeze(0)
                if on_gpu:
                    image = image.cuda()
                teacher_output = teacher(image)
                mean_output = torch.mean(teacher_output, dim=[0, 2, 3])
                mean_outputs.append(mean_output)
             except Exception as e:
                print(f"Error processing training image {img_path} for mean: {e}")


    if not mean_outputs:
         print("ERROR: Failed to process any training images for mean!")
         # Use dummy statistics
         channel_mean = torch.zeros((1, out_channels, 1, 1))
         channel_std = torch.ones((1, out_channels, 1, 1))
         if on_gpu:
             channel_mean = channel_mean.cuda()
             channel_std = channel_std.cuda()
    else:
        channel_mean = torch.mean(torch.stack(mean_outputs), dim=0)
        channel_mean = channel_mean[None, :, None, None]

        mean_distances = []
        with torch.no_grad():
             # Limit training images for speed if too many
            sample_images = train_images[:min(50, len(train_images))] # Use a subset
            for img_path in tqdm(sample_images, desc="Computing std"):
                try:
                    image = Image.open(img_path).convert('RGB')
                    image = transform(image).unsqueeze(0)
                    if on_gpu:
                        image = image.cuda()
                    teacher_output = teacher(image)
                    distance = (teacher_output - channel_mean) ** 2
                    mean_distance = torch.mean(distance, dim=[0, 2, 3])
                    mean_distances.append(mean_distance)
                except Exception as e:
                    print(f"Error processing training image {img_path} for std: {e}")

        if not mean_distances:
            print("ERROR: Failed to process any training images for std!")
            # Use dummy statistics (should already be set, but just in case)
            channel_std = torch.ones((1, out_channels, 1, 1))
            if on_gpu:
                channel_std = channel_std.cuda()
        else:
            channel_var = torch.mean(torch.stack(mean_distances), dim=0)
            channel_var = channel_var[None, :, None, None]
            channel_std = torch.sqrt(channel_var)


# Define the path where you want to save the combined model
combined_model_path = os.path.join(MODEL_PATH, 'efficientad_combined_model.pth')

# Create a dictionary containing the models and any necessary parameters
# Include channel_mean and channel_std if they are needed for inference
# Include quantiles if they are needed for inference
model_state = {
    'teacher': teacher.state_dict(),
    'student': student.state_dict(),
    'autoencoder': autoencoder.state_dict(),
    'channel_mean': channel_mean.cpu(), # Move to CPU before saving if it's on GPU
    'channel_std': channel_std.cpu(),   # Move to CPU before saving if it's on GPU
    'q_st_start': q_st_start.cpu(),     # Move to CPU before saving if it's on GPU
    'q_st_end': q_st_end.cpu(),         # Move to CPU before saving if it's on GPU
    'q_ae_start': q_ae_start.cpu(),     # Move to CPU before saving if it's on GPU
    'q_ae_end': q_ae_end.cpu(),         # Move to CPU before saving if it's on GPU
    'out_channels': out_channels, # Include constants needed for inference
    'image_size': image_size,     # Include constants needed for inference
    'transform': transform        # Include the transform if it's part of the model logic
}

# Save the combined state dictionary
torch.save(model_state, combined_model_path)

print(f"Model saved successfully to: {combined_model_path}")

Loading models and quantiles for saving...
Successfully loaded final models
Loading quantiles for saving...
Successfully loaded quantiles
Loading training images for normalization (if needed)...
Model saved successfully to: /content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood/efficientad_combined_model.pth


In [ ]:
# prompt: Modeli yükleyip test defect ve good klasöründen beşer resme sonuç verdirtmek istiyorum
import random
import os # Ensure os is imported
import torch # Ensure torch is imported
import numpy as np # Ensure numpy is imported
import matplotlib.pyplot as plt # Ensure matplotlib is imported
from PIL import Image # Ensure PIL is imported
from glob import glob # Ensure glob is imported
from tqdm import tqdm # Ensure tqdm is imported
from torchvision import transforms # Ensure transforms is imported
import tifffile # Ensure tifffile is imported
import cv2 # Ensure cv2 is imported
import traceback # Import the traceback module


# Fix for PyTorch 2.6+ pickle loading issue - Ensure this is also in the saving cell
from torch.serialization import add_safe_globals
add_safe_globals([
    'torch.nn.modules.container.Sequential',
    'torch.nn.modules.conv.Conv2d',
    'torch.nn.modules.activation.ReLU',
    'torch.nn.modules.pooling.AvgPool2d',
    'torch.nn.modules.linear.Linear',
    'torch.nn.modules.normalization.GroupNorm',
    'torch.nn.modules.activation.GELU',
    'torchvision.models.resnet.Bottleneck',
    'collections.OrderedDict'
])


# Define paths again to make sure they are available in this cell
# Redefine DATASET_PATH
DATASET_PATH = "/content/drive/MyDrive/wood_otsu/wood/test" # Define DATASET_PATH here
MODEL_PATH = "/content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood" # Replace with the correct model path
TRAIN_DIR = "/content/drive/MyDrive/Wood_Dataset/wood/train"  # Path to training images
GT_DIR = "/content/drive/MyDrive/wood_otsu/wood/ground_truth"  # Ground truth directory

# Paths for testing specific images
GOOD_IMAGES_DIR = os.path.join(DATASET_PATH, "good") # Use DATASET_PATH
DEFECT_IMAGES_DIR = os.path.join(DATASET_PATH, "defect") # Use DATASET_PATH
OUTPUT_DIR_SPECIFIC = "/content/drive/MyDrive/EfficientAD_/result-specific_images/png" # Separate output for specific tests

# Contour threshold for anomaly detection - Updated to 0.250
CONTOUR_THRESHOLD = 0.130 # Ensure this is defined


# Number of images to test from each category
NUM_IMAGES_TO_TEST = 5

print(f"Testing {NUM_IMAGES_TO_TEST} good images from: {GOOD_IMAGES_DIR}")
print(f"Testing {NUM_IMAGES_TO_TEST} defect images from: {DEFECT_IMAGES_DIR}")
os.makedirs(os.path.join(OUTPUT_DIR_SPECIFIC, "grid_views"), exist_ok=True)

# Find good images
good_image_patterns = [
    os.path.join(GOOD_IMAGES_DIR, "*.jpg"),
    os.path.join(GOOD_IMAGES_DIR, "*.png")
]
all_good_images = []
for pattern in good_image_patterns:
    all_good_images.extend(glob(pattern))

# Find defect images
defect_image_patterns = [
    os.path.join(DEFECT_IMAGES_DIR, "*.jpg"),
    os.path.join(DEFECT_IMAGES_DIR, "*.png")
]
all_defect_images = []
for pattern in defect_image_patterns:
    all_defect_images.extend(glob(pattern))

# Select a subset of images
selected_good_images = random.sample(all_good_images, min(NUM_IMAGES_TO_TEST, len(all_good_images)))
selected_defect_images = random.sample(all_defect_images, min(NUM_IMAGES_TO_TEST, len(all_defect_images)))

print(f"Selected {len(selected_good_images)} good images.")
print(f"Selected {len(selected_defect_images)} defect images.")

# Combine selected images
selected_test_images = selected_good_images + selected_defect_images

# Constants (ensure these are also defined or loaded)
on_gpu = torch.cuda.is_available() # Ensure on_gpu is defined
out_channels = 384 # Ensure out_channels is defined
image_size = 256 # Ensure image_size is defined

# Default transform (ensure this is also defined or loaded)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load models and quantiles (ensure this block is present and working)
print("Loading models and quantiles...")
try:
    # Try loading with weights_only=False to handle PyTorch 2.6 changes
    teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_final.pth'), weights_only=False)
    student = torch.load(os.path.join(MODEL_PATH, 'student_final.pth'), weights_only=False)
    autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_final.pth'), weights_only=False)
    print("Successfully loaded final models")
except Exception as e:
    print(f"Error loading final models: {e}")
    try:
        # Try temporary models
        print("Trying temporary models...")
        teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_tmp.pth'), weights_only=False)
        student = torch.load(os.path.join(MODEL_PATH, 'student_tmp.pth'), weights_only=False)
        autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_tmp.pth'), weights_only=False)
        print("Successfully loaded temporary models")
    except Exception as e:
        print(f"Error loading temporary models: {e}")
        raise


# Load quantiles
print("Loading quantiles...")
try:
    quantiles = torch.load(os.path.join(MODEL_PATH, 'map_quantiles_final.pth'), weights_only=False)
    q_st_start = torch.tensor(quantiles['q_st_start']) # Convert to tensor
    q_st_end = torch.tensor(quantiles['q_st_end'])  # Convert to tensor
    q_ae_start = torch.tensor(quantiles['q_ae_start'])  # Convert to tensor
    q_ae_end = torch.tensor(quantiles['q_ae_end'])  # Convert to tensor
    print("Successfully loaded quantiles")
except Exception as e:
    print(f"Error loading quantiles: {e}")
    raise

if on_gpu:
    teacher = teacher.cuda()
    student = student.cuda()
    autoencoder = autoencoder.cuda()
    q_st_start = q_st_start.cuda()
    q_st_end = q_st_end.cuda()
    q_ae_start = q_ae_start.cuda()
    q_ae_end = q_ae_end.cuda()

# Set models to evaluation mode (important for consistent behavior)
teacher.eval()
student.eval()
autoencoder.eval()


# Load train data for computing teacher normalization if not already computed
# This part is crucial to ensure channel_mean and channel_std are defined
print("Loading training images for normalization (if needed)...")

train_patterns = [
    os.path.join(TRAIN_DIR, "*.jpg"),
    os.path.join(TRAIN_DIR, "*.png"),
    os.path.join(TRAIN_DIR, "**", "*.jpg"),
    os.path.join(TRAIN_DIR, "**", "*.png"),
    os.path.join(TRAIN_DIR, "good", "*.jpg"),
    os.path.join(TRAIN_DIR, "good", "*.png")
]

train_images = []
for pattern in train_patterns:
    found_images = glob(pattern, recursive=True)
    if found_images:
        train_images.extend(found_images)

if not train_images:
    print("WARNING: No training images found. Using dummy statistics.")
    # Create some dummy statistics that should work reasonably well
    channel_mean = torch.zeros((1, out_channels, 1, 1))
    channel_std = torch.ones((1, out_channels, 1, 1))
    if on_gpu:
        channel_mean = channel_mean.cuda()
        channel_std = channel_std.cuda()
else:
    # If we have training images, compute normalization statistics
    print("Computing teacher normalization...")
    mean_outputs = []
    with torch.no_grad():
        # Limit training images for speed if too many
        sample_images = train_images[:min(50, len(train_images))] # Use a subset
        for img_path_norm in tqdm(sample_images, desc="Computing mean"): # Use a different variable name
             try:
                image = Image.open(img_path_norm).convert('RGB')
                image = transform(image).unsqueeze(0)
                if on_gpu:
                    image = image.cuda()
                teacher_output = teacher(image)
                mean_output = torch.mean(teacher_output, dim=[0, 2, 3])
                mean_outputs.append(mean_output)
             except Exception as e:
                print(f"Error processing training image {img_path_norm} for mean: {e}")


    if not mean_outputs:
         print("ERROR: Failed to process any training images for mean!")
         # Use dummy statistics
         channel_mean = torch.zeros((1, out_channels, 1, 1))
         channel_std = torch.ones((1, out_channels, 1, 1))
         if on_gpu:
             channel_mean = channel_mean.cuda()
             channel_std = channel_std.cuda()
    else:
        channel_mean = torch.mean(torch.stack(mean_outputs), dim=0)
        channel_mean = channel_mean[None, :, None, None]

        mean_distances = []
        with torch.no_grad():
             # Limit training images for speed if too many
            sample_images = train_images[:min(50, len(train_images))] # Use a subset
            for img_path_norm in tqdm(sample_images, desc="Computing std"): # Use a different variable name
                try:
                    image = Image.open(img_path_norm).convert('RGB')
                    image = transform(image).unsqueeze(0)
                    if on_gpu:
                        image = image.cuda()
                    teacher_output = teacher(image)
                    distance = (teacher_output - channel_mean) ** 2
                    mean_distance = torch.mean(distance, dim=[0, 2, 3])
                    mean_distances.append(mean_distance)
                except Exception as e:
                    print(f"Error processing training image {img_path_norm} for std: {e}")

        if not mean_distances:
            print("ERROR: Failed to process any training images for std!")
            # Use dummy statistics (should already be set, but just in case)
            channel_std = torch.ones((1, out_channels, 1, 1))
            if on_gpu:
                channel_std = channel_std.cuda()
        else:
            channel_var = torch.mean(torch.stack(mean_distances), dim=0)
            channel_var = channel_var[None, :, None, None]
            channel_std = torch.sqrt(channel_var)


# Helper functions for contour detection and border removal (ensure these are defined or copied)
def filter_contours(contours, image_shape, min_area=100, border_margin=30):
    """Filter contours based on size and position"""
    h, w = image_shape[:2]
    filtered = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, cw, ch = cv2.boundingRect(cnt)
        if area > min_area and border_margin < x < w-border_margin and border_margin < y < h-border_margin:
            filtered.append(cnt)
    return filtered

def remove_border_artifacts(anomaly_map, border_px=30):
    """Remove border artifacts from anomaly map"""
    h, w = anomaly_map.shape
    mask = np.zeros_like(anomaly_map)
    mask[border_px:h-border_px, border_px:w-border_px] = 1
    return anomaly_map * mask


if not selected_test_images:
    print("No specific test images found.")
else:
    print("Starting inference on selected images...")
    specific_results = []

    with torch.no_grad():
        for img_path in tqdm(selected_test_images, desc="Processing selected test images"):
            try:
                # Extract class and filename
                # This line was causing the error, ensure DATASET_PATH is defined above
                relative_path = os.path.relpath(img_path, DATASET_PATH)
                parts = relative_path.split(os.sep)

                if len(parts) > 1:
                    class_name = parts[-2] # Correctly get class name from the parent folder
                    filename = parts[-1]   # Correctly get filename
                else:
                    class_name = "unknown" # Default to unknown if path structure is unexpected
                    filename = parts[-1]

                img_id = os.path.splitext(filename)[0]

                # Create output directory for this class in the specific output location
                class_output_dir = os.path.join(OUTPUT_DIR_SPECIFIC, class_name)
                os.makedirs(class_output_dir, exist_ok=True)

                # Load and preprocess image
                image = Image.open(img_path).convert('RGB')
                orig_width, orig_height = image.size
                image_tensor = transform(image).unsqueeze(0)

                if on_gpu:
                    image_tensor = image_tensor.cuda()

                # Run EfficientAD inference
                teacher_output = teacher(image_tensor)
                # Use the globally computed channel_mean and channel_std
                teacher_output = (teacher_output - channel_mean) / channel_std
                student_output = student(image_tensor)
                autoencoder_output = autoencoder(image_tensor)

                map_st = torch.mean((teacher_output - student_output[:, :out_channels])**2,
                                   dim=1, keepdim=True)
                map_ae = torch.mean((autoencoder_output - student_output[:, out_channels:])**2,
                                   dim=1, keepdim=True)

                # Apply quantile normalization
                # Use the globally loaded quantiles
                map_st = 0.1 * (map_st - q_st_start) / (q_st_end - q_st_start)
                map_ae = 0.1 * (map_ae - q_ae_start) / (q_ae_end - q_ae_start)

                # Combine maps
                map_combined = 0.5 * map_st + 0.5 * map_ae

                # Pad and resize back to original dimensions
                map_combined = torch.nn.functional.pad(map_combined, (4, 4, 4, 4))
                map_combined = torch.nn.functional.interpolate(
                    map_combined, (orig_height, orig_width), mode='bilinear')

                # Convert to numpy
                anomaly_map = map_combined[0, 0].cpu().numpy()

                # Save as tiff file (for full precision)
                tiff_path = os.path.join(class_output_dir, f"{img_id}.tiff")
                tifffile.imwrite(tiff_path, anomaly_map)

                # Load original image for overlay
                orig_img = np.array(Image.open(img_path).convert('RGB'))

                # Remove border artifacts for scoring
                anomaly_map_masked = remove_border_artifacts(anomaly_map.copy(), border_px=40)
                max_score = np.max(anomaly_map_masked)

                # Create overlay with contours
                overlay_img = orig_img.copy()

                # Create binary mask for contours
                binary_mask = (anomaly_map_masked > CONTOUR_THRESHOLD).astype(np.uint8) * 255
                contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                contours = filter_contours(contours, orig_img.shape, min_area=200, border_margin=40)

                # Draw contours on overlay
                overlay_img_cv = cv2.cvtColor(overlay_img, cv2.COLOR_RGB2BGR)
                cv2.drawContours(overlay_img_cv, contours, -1, (0, 0, 255), 2)  # Red contours

                # Add prediction text
                prediction = "Anomali" if max_score > CONTOUR_THRESHOLD else "Normal"
                cv2.putText(overlay_img_cv, f"Tahmin: {prediction} ({max_score:.3f})",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

                # Convert back to RGB for matplotlib
                overlay_img = cv2.cvtColor(overlay_img_cv, cv2.COLOR_BGR2RGB)

                # Create heatmap visualization
                tiff_norm = cv2.normalize(anomaly_map, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
                tiff_colored = cv2.applyColorMap(tiff_norm, cv2.COLORMAP_JET)
                tiff_colored = cv2.cvtColor(tiff_colored, cv2.COLOR_BGR2RGB)


                # Try to load ground truth mask if available
                gt_mask = None
                # Modify potential_gt_paths to specifically look in the defect ground_truth folder if class_name is defect
                if class_name == 'defect':
                     potential_gt_paths = [
                        os.path.join(GT_DIR, class_name, f"{img_id}_mask.png"),
                        os.path.join(GT_DIR, class_name, f"{img_id}_mask.jpg"),
                        os.path.join(GT_DIR, "defect", f"{img_id}_mask.png"), # Look directly in 'defect' if it's a subfolder
                        os.path.join(GT_DIR, "defect", f"{img_id}_mask.jpg"),
                        os.path.join(GT_DIR, f"{img_id}_mask.png"), # Also check root of GT_DIR
                        os.path.join(GT_DIR, f"{img_id}_mask.jpg")
                    ]
                else: # For 'good' or 'unknown', create a blank mask
                    potential_gt_paths = [] # No ground truth mask expected for 'good' images

                for gt_path in potential_gt_paths:
                    if os.path.exists(gt_path):
                        try:
                            # Ensure GT mask is grayscale and convert to 3 channel for display
                            gt_mask_raw = np.array(Image.open(gt_path).convert('L'))
                            gt_mask = cv2.cvtColor(gt_mask_raw, cv2.COLOR_GRAY2RGB)
                            print(f"Found ground truth mask at: {gt_path}")
                            break # Found a valid mask, stop searching
                        except Exception as gt_e:
                             print(f"Error loading ground truth mask {gt_path}: {gt_e}")
                             gt_mask = None # Reset if loading fails


                # If ground truth mask not found or loading failed, create a blank one
                if gt_mask is None:
                    print(f"Ground truth mask not found for {img_path}. Creating blank mask.")
                    gt_mask = np.ones_like(orig_img) * 255
                    # Optionally, draw a subtle border or text to indicate it's a blank GT
                    cv2.putText(gt_mask, "No GT Mask", (10, orig_height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)


                # Create grid visualization (Original, Anomaly Map, Overlay, Ground Truth)
                plt.figure(figsize=(15, 5))

                # Original Image
                plt.subplot(1, 4, 1)
                plt.imshow(orig_img)
                plt.title(f"{filename} ({class_name}) - Original")
                plt.axis('off')

                # Anomaly Map
                plt.subplot(1, 4, 2)
                plt.imshow(tiff_colored)
                plt.title(f"Anomaly Map (.tiff) - Max: {max_score:.3f}")
                plt.axis('off')

                # Overlay
                plt.subplot(1, 4, 3)
                plt.imshow(overlay_img)
                plt.title(f"Overlay (Tahmin: {prediction})")
                plt.axis('off')

                # Ground Truth
                plt.subplot(1, 4, 4)
                plt.imshow(gt_mask)
                plt.title("Ground Truth")
                plt.axis('off')

                # Save grid visualization to the specific output directory
                grid_path_specific = os.path.join(OUTPUT_DIR_SPECIFIC, "grid_views", f"{class_name}_{img_id}_grid.png")
                plt.tight_layout()
                plt.savefig(grid_path_specific, dpi=150, bbox_inches='tight')
                plt.close()

                # Store result for summary
                specific_results.append({
                    'path': img_path,
                    'class': class_name,
                    'filename': filename,
                    'img_id': img_id,
                    'score': max_score,
                    'prediction': prediction,
                    'has_contours': len(contours) > 0,
                    'grid_path': grid_path_specific # Use the path from the specific output
                })

            except Exception as e:
                print(f"Error processing specific test image {img_path}: {e}")
                traceback.print_exc() # Use traceback.print_exc() here

    # Create a separate summary report for specific tests
    try:
        print("Creating specific test summary report...")
        report_path_specific = os.path.join(OUTPUT_DIR_SPECIFIC, "specific_test_summary_report.html")
        with open(report_path_specific, 'w') as f:
            f.write("<html><head>")
            f.write("<style>")
            f.write("body { font-family: Arial, sans-serif; margin: 20px; }")
            f.write("h1 { color: #333; }")
            f.write("table { border-collapse: collapse; width: 100%; }")
            f.write("th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }")
            f.write("th { background-color: #f2f2f2; }")
            f.write("tr:nth-child(even) { background-color: #f9f9f9; }")
            f.write("img { max-width: 800px; border: 1px solid #ddd; }") # Smaller image size for specific tests
            f.write("</style>")
            f.write("</head><body>")
            f.write("<h1>Specific Anomaly Detection Test Results</h1>")

            # Summary statistics
            good_samples_specific = [r for r in specific_results if r['class'] == 'good']
            defect_samples_specific = [r for r in specific_results if r['class'] != 'good']

            f.write(f"<p>Total samples tested: {len(specific_results)}</p>")
            f.write(f"<p>Good samples tested: {len(good_samples_specific)}</p>")
            f.write(f"<p>Defect samples tested: {len(defect_samples_specific)}</p>")

            # Table of results
            f.write("<h2>Results Table</h2>")
            f.write("<table>")
            f.write("<tr><th>ID</th><th>Class</th><th>Score</th><th>Prediction</th><th>Grid View</th></tr>")

            for r in sorted(specific_results, key=lambda x: x['score'], reverse=True):
                f.write(f"<tr>")
                f.write(f"<td>{r['img_id']}</td>")
                f.write(f"<td>{r['class']}</td>")
                f.write(f"<td>{r['score']:.4f}</td>")
                f.write(f"<td>{r['prediction']}</td>")
                # Link to the grid view image
                f.write(f"<td><a href='{os.path.relpath(r['grid_path'], OUTPUT_DIR_SPECIFIC)}' target='_blank'>View Grid</a></td>")
                f.write(f"</tr>")

            f.write("</table>")

            # Include the grid views directly below the table
            f.write("<h2>Grid Views</h2>")
            for r in sorted(specific_results, key=lambda x: x['class']): # Sort by class for better grouping
                 f.write(f"<h3>{r['filename']} (Class: {r['class']}, Score: {r['score']:.4f})</h3>")
                 f.write(f"<img src='{os.path.relpath(r['grid_path'], OUTPUT_DIR_SPECIFIC)}'>")
                 f.write("<hr>")


            f.write("</body></html>")

        print(f"Specific test summary report created at: {report_path_specific}")
    except Exception as e:
        print(f"Error creating specific test summary report: {e}")


    print(f"Specific test results saved to {OUTPUT_DIR_SPECIFIC}")
    print("Specific tests complete!")

Testing 5 good images from: /content/drive/MyDrive/wood_otsu/wood/test/good
Testing 5 defect images from: /content/drive/MyDrive/wood_otsu/wood/test/defect
Selected 5 good images.
Selected 5 defect images.
Loading models and quantiles...
Successfully loaded final models
Loading quantiles...
Successfully loaded quantiles
Loading training images for normalization (if needed)...
Starting inference on selected images...


Processing selected test images:   0%|          | 0/10 [00:00<?, ?it/s]

Ground truth mask not found for /content/drive/MyDrive/wood_otsu/wood/test/good/72.jpg. Creating blank mask.


Processing selected test images:  10%|█         | 1/10 [00:00<00:04,  1.98it/s]

Ground truth mask not found for /content/drive/MyDrive/wood_otsu/wood/test/good/140.jpg. Creating blank mask.


...

Found ground truth mask at: /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/100000018_mask.jpg


Processing selected test images:  90%|█████████ | 9/10 [00:08<00:01,  1.10s/it]

Found ground truth mask at: /content/drive/MyDrive/wood_otsu/wood/ground_truth/defect/100000007_mask.jpg


Processing selected test images: 100%|██████████| 10/10 [00:10<00:00,  1.05s/it]

Creating specific test summary report...
Specific test summary report created at: /content/drive/MyDrive/EfficientAD_/result-specific_images/png/specific_test_summary_report.html
Specific test results saved to /content/drive/MyDrive/EfficientAD_/result-specific_images/png
Specific tests complete!


In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm import tqdm
from torchvision import transforms
import tifffile
import cv2
from sklearn.metrics import roc_curve # Import roc_curve here as well for clarity

# Fix for PyTorch 2.6+ pickle loading issue
from torch.serialization import add_safe_globals
add_safe_globals([
    'torch.nn.modules.container.Sequential',
    'torch.nn.modules.conv.Conv2d',
    'torch.nn.modules.activation.ReLU',
    'torch.nn.modules.pooling.AvgPool2d',
    'torch.nn.modules.linear.Linear',
    'torch.nn.modules.normalization.GroupNorm',
    'torch.nn.modules.activation.GELU',
    'torchvision.models.resnet.Bottleneck',
    'collections.OrderedDict'
])

# Paths - UPDATE THESE TO MATCH YOUR ACTUAL DIRECTORIES

DATASET_PATH = "/content/drive/MyDrive/wood_otsu/wood/test"
MODEL_PATH = "/content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood"
OUTPUT_DIR = "/content/drive/MyDrive/EfficientAD_/result-13Mayıs_/png"
TRAIN_DIR = "/content/drive/MyDrive/Wood_Dataset/wood/train"  # Path to training images
GT_DIR = "/content/drive/MyDrive/wood_otsu/wood/ground_truth"  # Ground truth directory

# Contour threshold for anomaly detection - Updated to 0.250
CONTOUR_THRESHOLD = 0.0381
# Print paths for verification
print(f"Dataset Path: {DATASET_PATH}")
print(f"Model Path: {MODEL_PATH}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Train Directory: {TRAIN_DIR}")
print(f"Ground Truth Directory: {GT_DIR}")
print(f"Contour Threshold: {CONTOUR_THRESHOLD}")

# Check if directories exist
for path, name in [(DATASET_PATH, "Dataset"), (MODEL_PATH, "Model"), (TRAIN_DIR, "Train"), (GT_DIR, "Ground Truth")]:
    if not os.path.exists(path):
        print(f"WARNING: {name} path does not exist: {path}")
    else:
        print(f"{name} path exists: {path}")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "grid_views"), exist_ok=True)

# Constants
on_gpu = torch.cuda.is_available()
out_channels = 384
image_size = 256

# Default transform for testing
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Helper functions for contour detection and border removal
def filter_contours(contours, image_shape, min_area=100, border_margin=30):
    """Filter contours based on size and position"""
    h, w = image_shape[:2]
    filtered = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, cw, ch = cv2.boundingRect(cnt)
        if area > min_area and border_margin < x < w-border_margin and border_margin < y < h-border_margin:
            filtered.append(cnt)
    return filtered

def remove_border_artifacts(anomaly_map, border_px=30):
    """Remove border artifacts from anomaly map"""
    h, w = anomaly_map.shape
    mask = np.zeros_like(anomaly_map)
    mask[border_px:h-border_px, border_px:w-border_px] = 1
    return anomaly_map * mask

print("Loading models and quantiles...")
try:
    # Try loading with weights_only=False to handle PyTorch 2.6 changes
    teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_final.pth'), weights_only=False)
    student = torch.load(os.path.join(MODEL_PATH, 'student_final.pth'), weights_only=False)
    autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_final.pth'), weights_only=False)
    print("Successfully loaded final models")
except Exception as e:
    print(f"Error loading final models: {e}")
    try:
        # Try temporary models
        print("Trying temporary models...")
        teacher = torch.load(os.path.join(MODEL_PATH, 'teacher_tmp.pth'), weights_only=False)
        student = torch.load(os.path.join(MODEL_PATH, 'student_tmp.pth'), weights_only=False)
        autoencoder = torch.load(os.path.join(MODEL_PATH, 'autoencoder_tmp.pth'), weights_only=False)
        print("Successfully loaded temporary models")
    except Exception as e:
        print(f"Error loading temporary models: {e}")
        raise

# Load quantiles
print("Loading quantiles...")
try:
    quantiles = torch.load(os.path.join(MODEL_PATH, 'map_quantiles_final.pth'), weights_only=False)
    q_st_start = torch.tensor(quantiles['q_st_start']) # Convert to tensor
    q_st_end = torch.tensor(quantiles['q_st_end'])  # Convert to tensor
    q_ae_start = torch.tensor(quantiles['q_ae_start'])  # Convert to tensor
    q_ae_end = torch.tensor(quantiles['q_ae_end'])  # Convert to tensor
    print("Successfully loaded quantiles")
except Exception as e:
    print(f"Error loading quantiles: {e}")
    raise

if on_gpu:
    teacher = teacher.cuda()
    student = student.cuda()
    autoencoder = autoencoder.cuda()
    q_st_start = q_st_start.cuda()
    q_st_end = q_st_end.cuda()
    q_ae_start = q_ae_start.cuda()
    q_ae_end = q_ae_end.cuda()

# Set models to evaluation mode
teacher.eval()
student.eval()
autoencoder.eval()

# Load train data for computing teacher normalization
print("Loading training images for normalization...")

# Try different patterns to locate training images
train_patterns = [
    os.path.join(TRAIN_DIR, "*.jpg"),
    os.path.join(TRAIN_DIR, "*.png"),
    os.path.join(TRAIN_DIR, "**", "*.jpg"),
    os.path.join(TRAIN_DIR, "**", "*.png"),
    os.path.join(TRAIN_DIR, "good", "*.jpg"),
    os.path.join(TRAIN_DIR, "good", "*.png")
]

train_images = []
for pattern in train_patterns:
    found_images = glob(pattern, recursive=True)
    if found_images:
        print(f"Found {len(found_images)} images with pattern: {pattern}")
        train_images.extend(found_images)

if not train_images:
    print("ERROR: No training images found! Checking parent directories...")
    # Try to look in parent directories
    parent_dir = os.path.dirname(TRAIN_DIR)
    for pattern in [os.path.join(parent_dir, "**", "*.jpg"), os.path.join(parent_dir, "**", "*.png")]:
        found_images = glob(pattern, recursive=True)
        if found_images:
            print(f"Found {len(found_images)} images in parent directory with pattern: {pattern}")
            train_images.extend(found_images)

# Print first few train images for verification
if train_images:
    print(f"Total training images found: {len(train_images)}")
    print("First few training images:")
    for i in range(min(5, len(train_images))):
        print(f"  - {train_images[i]}")
else:
    print("ERROR: Could not find any training images! Cannot proceed without training images.")
    # Instead of crashing, we'll use pre-computed statistics from the quantiles
    print("Using pre-computed statistics from quantiles to continue...")
    # Create some dummy statistics that should work reasonably well
    channel_mean = torch.zeros((1, out_channels, 1, 1))
    channel_std = torch.ones((1, out_channels, 1, 1))
    if on_gpu:
        channel_mean = channel_mean.cuda()
        channel_std = channel_std.cuda()

# If we have training images, compute normalization statistics
if train_images:
    # If there are too many images, use just a subset for normalization
    if len(train_images) > 50:
        import random
        train_images = random.sample(train_images, 50)
        print(f"Using random subset of 50 images for normalization")

    # Compute teacher normalization
    print("Computing teacher normalization...")
    mean_outputs = []
    with torch.no_grad():
        for img_path in tqdm(train_images):
            try:
                image = Image.open(img_path).convert('RGB')
                image = transform(image).unsqueeze(0)
                if on_gpu:
                    image = image.cuda()
                teacher_output = teacher(image)
                mean_output = torch.mean(teacher_output, dim=[0, 2, 3])
                mean_outputs.append(mean_output)
            except Exception as e:
                print(f"Error processing training image {img_path}: {e}")

    if not mean_outputs:
        print("ERROR: Failed to process any training images!")
        # Use dummy statistics
        channel_mean = torch.zeros((1, out_channels, 1, 1))
        channel_std = torch.ones((1, out_channels, 1, 1))
        if on_gpu:
            channel_mean = channel_mean.cuda()
            channel_std = channel_std.cuda()
    else:
        channel_mean = torch.mean(torch.stack(mean_outputs), dim=0)
        channel_mean = channel_mean[None, :, None, None]

        mean_distances = []
        with torch.no_grad():
            for img_path in tqdm(train_images):
                try:
                    image = Image.open(img_path).convert('RGB')
                    image = transform(image).unsqueeze(0)
                    if on_gpu:
                        image = image.cuda()
                    teacher_output = teacher(image)
                    distance = (teacher_output - channel_mean) ** 2
                    mean_distance = torch.mean(distance, dim=[0, 2, 3])
                    mean_distances.append(mean_distance)
                except Exception as e:
                    print(f"Error processing training image distance {img_path}: {e}")

        channel_var = torch.mean(torch.stack(mean_distances), dim=0)
        channel_var = channel_var[None, :, None, None]
        channel_std = torch.sqrt(channel_var)

# Find all test images (both good and defect)
print("Looking for test images...")
test_patterns = [
    os.path.join(DATASET_PATH, "**", "*.jpg"),
    os.path.join(DATASET_PATH, "**", "*.png"),
    os.path.join(DATASET_PATH, "*.jpg"),
    os.path.join(DATASET_PATH, "*.png"),
    os.path.join(DATASET_PATH, "good", "*.jpg"),
    os.path.join(DATASET_PATH, "good", "*.png"),
    os.path.join(DATASET_PATH, "defect", "*.jpg"),
    os.path.join(DATASET_PATH, "defect", "*.png")
]

test_images = []
for pattern in test_patterns:
    found_images = glob(pattern, recursive=True)
    if found_images:
        print(f"Found {len(found_images)} test images with pattern: {pattern}")
        test_images.extend(found_images)

# Remove duplicates
test_images = list(set(test_images))

if not test_images:
    print("ERROR: No test images found!")
    exit(1)

print(f"Found {len(test_images)} total test images.")
print("First few test images:")
for i in range(min(5, len(test_images))):
    print(f"  - {test_images[i]}")

# Run inference on all test images
print("Starting inference...")
results = []
y_true = [] # Initialize list to store true labels
y_score = [] # Initialize list to store anomaly scores

with torch.no_grad():
    for img_path in tqdm(test_images, desc="Processing test images"):
        try:
            # Extract class and filename
            relative_path = os.path.relpath(img_path, DATASET_PATH)
            parts = relative_path.split(os.sep)

            if len(parts) > 1:
                class_name = parts[0]
                filename = parts[1]
                # Determine true label (0 for good, 1 for defect)
                true_label = 0 if class_name == 'good' else 1
            else:
                class_name = "unknown"
                filename = parts[0]
                true_label = -1 # Indicate unknown or unable to determine class

            img_id = os.path.splitext(filename)[0]

            # Create output directory for this class
            class_output_dir = os.path.join(OUTPUT_DIR, class_name)
            os.makedirs(class_output_dir, exist_ok=True)

            # Load and preprocess image
            image = Image.open(img_path).convert('RGB')
            orig_width, orig_height = image.size
            image_tensor = transform(image).unsqueeze(0)

            if on_gpu:
                image_tensor = image_tensor.cuda()

            # Run EfficientAD inference
            teacher_output = teacher(image_tensor)
            teacher_output = (teacher_output - channel_mean) / channel_std
            student_output = student(image_tensor)
            autoencoder_output = autoencoder(image_tensor)

            map_st = torch.mean((teacher_output - student_output[:, :out_channels])**2,
                               dim=1, keepdim=True)
            map_ae = torch.mean((autoencoder_output - student_output[:, out_channels:])**2,
                               dim=1, keepdim=True)

            # Apply quantile normalization
            map_st = 0.1 * (map_st - q_st_start) / (q_st_end - q_st_start)
            map_ae = 0.1 * (map_ae - q_ae_start) / (q_ae_end - q_ae_start)

            # Combine maps
            map_combined = 0.5 * map_st + 0.5 * map_ae

            # Pad and resize back to original dimensions
            map_combined = torch.nn.functional.pad(map_combined, (4, 4, 4, 4))
            map_combined = torch.nn.functional.interpolate(
                map_combined, (orig_height, orig_width), mode='bilinear')

            # Convert to numpy
            anomaly_map = map_combined[0, 0].cpu().numpy()

            # Save as tiff file (for full precision)
            tiff_path = os.path.join(class_output_dir, f"{img_id}.tiff")
            tifffile.imwrite(tiff_path, anomaly_map)

            # Load original image for overlay
            orig_img = np.array(Image.open(img_path).convert('RGB'))

            # Remove border artifacts for scoring
            anomaly_map_masked = remove_border_artifacts(anomaly_map.copy(), border_px=40)
            max_score = np.max(anomaly_map_masked)

            # Append true label and anomaly score to lists
            if true_label != -1: # Only append if we could determine the true class
                y_true.append(true_label)
                y_score.append(max_score)

            # Create overlay with contours
            overlay_img = orig_img.copy()

            # Create binary mask for contours
            binary_mask = (anomaly_map_masked > CONTOUR_THRESHOLD).astype(np.uint8) * 255
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = filter_contours(contours, orig_img.shape, min_area=200, border_margin=40)

            # Draw contours on overlay
            overlay_img_cv = cv2.cvtColor(overlay_img, cv2.COLOR_RGB2BGR)
            cv2.drawContours(overlay_img_cv, contours, -1, (0, 0, 255), 2)  # Red contours

            # Add prediction text
            prediction = "Anomali" if max_score > CONTOUR_THRESHOLD else "Normal"
            cv2.putText(overlay_img_cv, f"Tahmin: {prediction} ({max_score:.3f})",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            # Convert back to RGB for matplotlib
            overlay_img = cv2.cvtColor(overlay_img_cv, cv2.COLOR_BGR2RGB)

            # Create heatmap visualization
            tiff_norm = cv2.normalize(anomaly_map, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
            tiff_colored = cv2.applyColorMap(tiff_norm, cv2.COLORMAP_JET)
            tiff_colored = cv2.cvtColor(tiff_colored, cv2.COLOR_BGR2RGB)

            # Try to load ground truth mask if available
            gt_mask = None
            potential_gt_paths = [
                os.path.join(GT_DIR, class_name, f"{img_id}_mask.png"),
                os.path.join(GT_DIR, class_name, f"{img_id}_mask.jpg"),
                os.path.join(GT_DIR, "defect", f"{img_id}_mask.png"),
                os.path.join(GT_DIR, "defect", f"{img_id}_mask.jpg"),
                os.path.join(GT_DIR, "ground_truth", class_name, f"{img_id}_mask.png"),
                os.path.join(GT_DIR, "ground_truth", class_name, f"{img_id}_mask.jpg"),
                os.path.join(GT_DIR, f"{img_id}_mask.png"),
                os.path.join(GT_DIR, f"{img_id}_mask.jpg")
            ]

            for gt_path in potential_gt_paths:
                if os.path.exists(gt_path):
                    gt_mask = np.array(Image.open(gt_path).convert('RGB'))
                    # print(f"Found ground truth mask at: {gt_path}") # Comment out to reduce output noise
                    break

            # If ground truth mask not found, create a blank one (for good images or missing masks)
            if gt_mask is None:
                gt_mask = np.ones_like(orig_img) * 255
                # Only center part is black if it's a good image
                if class_name == 'good':
                    cv2.rectangle(gt_mask, (0, 0), (orig_width, orig_height), (0, 0, 0), -1)
                    gt_mask[10:-10, 10:-10] = [255, 255, 255]


            # Create grid visualization (Original, Anomaly Map, Overlay, Ground Truth)
            plt.figure(figsize=(15, 5))

            # Original Image
            plt.subplot(1, 4, 1)
            plt.imshow(orig_img)
            plt.title(f"{filename} ({class_name}) - Original")
            plt.axis('off')

            # Anomaly Map
            plt.subplot(1, 4, 2)
            plt.imshow(tiff_colored)
            plt.title(f"Anomaly Map (.tiff) - Max: {max_score:.3f}")
            plt.axis('off')

            # Overlay
            plt.subplot(1, 4, 3)
            plt.imshow(overlay_img)
            plt.title(f"Overlay (Tahmin: {prediction})")
            plt.axis('off')

            # Ground Truth
            plt.subplot(1, 4, 4)
            plt.imshow(gt_mask)
            plt.title("Ground Truth")
            plt.axis('off')

            # Save grid visualization
            grid_path = os.path.join(OUTPUT_DIR, "grid_views", f"{class_name}_{img_id}_grid.png")
            plt.tight_layout()
            plt.savefig(grid_path, dpi=150, bbox_inches='tight')
            plt.close()

            # Store result for summary
            results.append({
                'path': img_path,
                'class': class_name,
                'filename': filename,
                'img_id': img_id,
                'score': max_score,
                'prediction': prediction,
                'has_contours': len(contours) > 0,
                'grid_path': grid_path,
                'true_label': true_label # Add true label to results
            })

        except Exception as e:
            print(f"Error processing test image {img_path}: {e}")
            import traceback
            traceback.print_exc()

print(f"All anomaly maps and visualizations saved to {OUTPUT_DIR}")

# Convert y_true and y_score to numpy arrays before calculating ROC curve
y_true = np.array(y_true)
y_score = np.array(y_score)

# Calculate ROC curve
try:
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    youden_index = tpr - fpr
    best_threshold = thresholds[np.argmax(youden_index)]
    print(f"Calculated best threshold using Youden's J statistic: {best_threshold:.4f}")
except Exception as e:
    print(f"Error calculating ROC curve or best threshold: {e}")


# Create summary report
try:
    print("Creating summary report...")
    report_path = os.path.join(OUTPUT_DIR, "summary_report.html")
    with open(report_path, 'w') as f:
        f.write("<html><head>")
        f.write("<style>")
        f.write("body { font-family: Arial, sans-serif; margin: 20px; }")
        f.write("h1 { color: #333; }")
        f.write("table { border-collapse: collapse; width: 100%; }")
        f.write("th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }")
        f.write("th { background-color: #f2f2f2; }")
        f.write("tr:nth-child(even) { background-color: #f9f9f9; }")
        f.write("img { max-width: 1000px; border: 1px solid #ddd; }")
        f.write("</style>")
        f.write("</head><body>")
        f.write("<h1>Anomaly Detection Results</h1>")

        # Summary statistics
        good_samples = [r for r in results if r['class'] == 'good']
        defect_samples = [r for r in results if r['class'] != 'good']

        f.write(f"<p>Total samples: {len(results)}</p>")
        f.write(f"<p>Good samples: {len(good_samples)}</p>")
        f.write(f"<p>Defect samples: {len(defect_samples)}</p>")
        if 'best_threshold' in locals(): # Check if best_threshold was calculated
             f.write(f"<p>Calculated Best Threshold (Youden's J): {best_threshold:.4f}</p>")


        # Table of results
        f.write("<h2>Results Table</h2>")
        f.write("<table>")
        f.write("<tr><th>ID</th><th>Class</th><th>Score</th><th>Prediction</th><th>True Label</th></tr>") # Added True Label column

        for r in sorted(results, key=lambda x: x['score'], reverse=True):
            f.write(f"<tr>")
            f.write(f"<td>{r['img_id']}</td>")
            f.write(f"<td>{r['class']}</td>")
            f.write(f"<td>{r['score']:.4f}</td>")
            f.write(f"<td>{r['prediction']}</td>")
            f.write(f"<td>{r['true_label']}</td>") # Display true label
            f.write(f"</tr>")

        f.write("</table>")

        # Images
        f.write("<h2>Top Anomalies</h2>")

        # Sort by score and take top 20
        top_anomalies = sorted(results, key=lambda x: x['score'], reverse=True)[:20]

        for r in top_anomalies:
            f.write(f"<h3>{r['filename']} (Class: {r['class']}, Score: {r['score']:.4f})</h3>")
            f.write(f"<img src='{os.path.relpath(r['grid_path'], OUTPUT_DIR)}'>")
            f.write("<hr>")

        f.write("</body></html>")

    print(f"Summary report created at: {report_path}")
except Exception as e:
    print(f"Error creating summary report: {e}")

print("Done!")

Dataset Path: /content/drive/MyDrive/wood_otsu/wood/test
Model Path: /content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood
Output Directory: /content/drive/MyDrive/EfficientAD_/result-13Mayıs_/png
Train Directory: /content/drive/MyDrive/Wood_Dataset/wood/train
Ground Truth Directory: /content/drive/MyDrive/wood_otsu/wood/ground_truth
Contour Threshold: 0.0381
Dataset path exists: /content/drive/MyDrive/wood_otsu/wood/test
Model path exists: /content/drive/MyDrive/EfficientAD_/Deneme 2 Ağırlıklar/trainings/mvtec_ad/wood
Ground Truth path exists: /content/drive/MyDrive/wood_otsu/wood/ground_truth
Loading models and quantiles...
Successfully loaded final models
Loading quantiles...
Successfully loaded quantiles
Loading training images for normalization...
ERROR: No training images found! Checking parent directories...
ERROR: Could not find any training images! Cannot proceed without training images.
Using pre-computed statistics from quantiles to continue...
Lo

Processing test images: 100%|██████████| 140/140 [03:08<00:00,  1.35s/it]

All anomaly maps and visualizations saved to /content/drive/MyDrive/EfficientAD_/result-13Mayıs_/png
Calculated best threshold using Youden's J statistic: 0.0381
Creating summary report...
Summary report created at: /content/drive/MyDrive/EfficientAD_/result-13Mayıs_/png/summary_report.html
Done!
